### Imports modules and libs

In [1]:
import pandas as pd 
import numpy as np
from ConvertFile import Convert_File_to_pd
from DataAnalyze import DataAnalyze
from AnalyzeDuplicatedData import AnalyzeDuplicatedData
from DataTransform import DataTransform
from LoadCsvs import LoadCsvs


### Transform CSVs to DFs

In [2]:
Data_frame=Convert_File_to_pd(file_name='Team_Attributes',name_table='Data',folder_name='ETL-Data-FIFA')

In [3]:
Df=Data_frame.convert_to_dataframe

### Exploratory analysis to investigate the possibility of data cleaning

In [4]:
#Imprimindo o DataFrame
Df

,Unnamed: 0,Team_Attributes
0,0,"{""id"": ""1"", ""team_fifa_api_id"": ""434"", ""team_a..."
1,1,"{""id"": ""2"", ""team_fifa_api_id"": ""434"", ""team_a..."
2,2,"{""id"": ""3"", ""team_fifa_api_id"": ""434"", ""team_a..."
3,3,"{""id"": ""4"", ""team_fifa_api_id"": ""77"", ""team_ap..."
4,4,"{""id"": ""5"", ""team_fifa_api_id"": ""77"", ""team_ap..."
...,...,...
1455,1455,"{""id"": ""1456"", ""team_fifa_api_id"": ""15005"", ""t..."
1456,1456,"{""id"": ""1457"", ""team_fifa_api_id"": ""15005"", ""t..."
1457,1457,"{""id"": ""1458"", ""team_fifa_api_id"": ""15005"", ""t..."
1458,1458,"{""id"": ""1"", ""team_fifa_api_id"": ""434"", ""team_a..."


In [5]:
#Classe responsável pelas análises exploratórias
Results_analysis=DataAnalyze(Df)

#### OBS

#### Detect NaNs

In [6]:
# No objetivo de não ferir a integridade de domínio quando estes dados forem carregados em um bd, o id não pode ter campos nulos ou NaN
Results_analysis.detect_nans(name_column='id')


KeyError: 'id'

In [ ]:
#Fazend isso para player_api_id 
Results_analysis.detect_nans(name_column='team_api_id')

In [ ]:
#Fazend isso para player_fifa_api_id	
Results_analysis.detect_nans(name_column='team_fifa_api_id')

#### Detect Data Duplicated

In [ ]:
#além disso, é necessário avaliar os registros duplicados quando a coluna não pode contê-los, como é o caso das colunas acima 
Result_dup_id=AnalyzeDuplicatedData(dataframe=Df,name_col_data='id')
Result_dup_team_api_id=AnalyzeDuplicatedData(dataframe=Df,name_col_data='team_api_id')
Result_dup_team_fifa_api_id=AnalyzeDuplicatedData(dataframe=Df,name_col_data='team_fifa_api_id')

In [ ]:
Result_dup_id.print_duplicated_data(table_name='Player')
Result_dup_team_api_id.print_duplicated_data(table_name='Player')
Result_dup_team_fifa_api_id.print_duplicated_data(table_name='Player')

In [ ]:
Result_dup_team_fifa_api_id.detect_values_data()

#### Identify Outliers

In [ ]:
#Como não existem colunas quantitativas, não é necessário avaliação de outliers


### Analyze  typing

In [ ]:
#analisando a tipagem esperada de forma geral
Df.info()


In [ ]:
#Como ids sob tipagem numérica são geralmente representados por inteiros,logo é necessário avaliar a coluna team_fifa_api_id por estar como float64

Results_analysis.analyze_typing_expected_number(expected_type_number='int',name_column='team_fifa_api_id')


In [ ]:
#avaliando se todos os dados seguem com a mesma quantidade de validações entre float e int, considerando também os NaNs

cont_float_and_int=len(Results_analysis.analyze_typing_expected_number(expected_type_number='int',name_column='team_fifa_api_id'))
print(cont_float_and_int+Df['team_fifa_api_id'].isna().sum()==(Df['team_fifa_api_id'].shape[0]))

In [ ]:
#confirmando se todos os dados estão vindo como str para as colunas team_long_name e team_short_name

Results_analysis.confirm_typing(name_column='team_long_name',type_expected='str')


In [ ]:
Results_analysis.confirm_typing(name_column='team_short_name',type_expected='str')

##Como as listas vieram vazias, todos os dados estão coerentes como str

### Data transform 

In [ ]:
#Instânciando o Df para a classe DataCleaning

Data_cleaning=DataTransform(dataframe_to_transform=Df)

#### Remove(or not) NaNs

In [ ]:
# partindo do pressuposto de que as colunas identificadoras (id, team_api_id e team_fifa_api_id)  não podem ter valores NaNs, é preciso remove-los
#Por possuirem valores distintos e não apresentam uma progressão lógica neste caso, além de serem propriamente identificadores

#Logo, assumiremos a remoção, haja vista que a proporção de NaNs próximos ou abaixo de 5 %  servirá como parâmetro para remoção 
Results_analysis.count_nans(name_columns=['id','team_api_id','team_fifa_api_id'])

In [ ]:
#Removendo os NaNs dos team_fifa_api_id

Data_cleaning.remove_nans(name_column=['team_fifa_api_id'])

#### Remove(or not) Duplicated


In [ ]:
#removendo os dados duplicados de team_fifa_api_id
#Como não haverá mais passos para tratamento de dados, esse último sera o Df transf
Data_Transf=Data_cleaning.remove_dup(name_column=['team_fifa_api_id'])

### Load CSV

In [ ]:
Df_load=LoadCsvs(dataframe=Data_Transf,new_folder='Load_csvs',name_dataframe='Team',folder_name=Data_frame.folder_name)

In [ ]:
Df_load.load_csv